# <center> MATH&ML-6. Математический анализ в контексте задачи оптимизации. Часть III

In [1]:
#Импорты
import numpy as np
import pandas as pd
import sympy
from sympy import Eq, solveset, Symbol, symbols, Interval, S, log, sin, cos, exp, diff, solve, N
# Аккуратный вывод формул
from sympy import init_printing

from scipy import optimize
from scipy.optimize import minimize

from sympy.calculus.util import function_range, continuous_domain

### Юнит 1. Введение

In [9]:
# Задание 1.1

x, w = symbols('x w')
func = -x**4 + 6*x**3 - 4*x**2 + 80
display(func.diff(x))
display(solve(func.diff(x)))

print('Ответ:',-4**4 + 6*4**3 - 4*4**2 + 80)

-4*x**3 + 18*x**2 - 8*x

[0, 1/2, 4]

Ответ: 144


In [26]:
#Задание 1.4
x, y, w = symbols('x y w')
func = x**2 + 2*y**2
g = x + y - 20

full_func = func + w*g
display(f'diff x: {full_func.diff(x)}')
display(f'diff y: {full_func.diff(y)}')
display(f'diff w: {full_func.diff(w)}')
xyw = solve([full_func.diff(x), full_func.diff(y), full_func.diff(w)], x, y, w)
display(xyw)
display(f'Ответ: {(xyw[x]**2 + 2*xyw[y]**2)*3}')

'diff x: w + 2*x'

'diff y: w + 4*y'

'diff w: x + y - 20'

{w: -80/3, x: 40/3, y: 20/3}

'Ответ: 800'